In [1]:
import json
import pandas as pd

filename = 'test.jsonl'
savename = 'test.csv'

data = []
with open(filename) as f:
    for line in f:
        data.append(json.loads(line))

data2 = []
for i in data:
    data2.append(i['text'])

data2 = pd.DataFrame(data2, columns=['text'])

labels = []
for i in data:
    labels.append(i['label'])
data2['label'] = labels
data2.to_csv(savename, index=False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3058682447.py, line 4)

In [11]:
import torch
import torch.nn as nn
from torch import Tensor
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [13]:
class SampleLoader(Dataset):
    def __init__(self, annotation_file):
        super().__init__()
        self.samples = pd.read_csv(annotation_file)
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        txt = self.samples.iloc[idx]['text']
        label = self.samples.iloc[idx]['label']
        return txt, label

In [15]:
trainer = SampleLoader(annotation_file='test.csv')
trainloader = DataLoader(dataset=trainer, batch_size=128, shuffle=False)

In [17]:
for i , (text, label) in enumerate(trainloader):
    print(text)
    print(label)
    break

('Police: Negotiators are now in contact with armed hostage-taker in Sydney situation; motive remains unknown - http://t.co/mp18E4DdjW', 'Muslims pray for the Friday prayer next to a sign reading #JeSuisCharlie in Saint-Etienne,France. By @JpKphotographer http://t.co/irukD1LedH', "Police of #Ferguson murdered a boy in cold blood. Now they're spinning the story & intimidating the community into silence.", "JUST IN: Canadian military bases being closed after soldier shot at Nat'l War Memorial, ongoing situation near Parliament - @WilliamsJon", "Cowardly attack on Charlie Hebdo's offices. The humourless fanatics can fuck off. Freedom of speech is more important than your feelings.", "Airbus plane operated by Lufthansa's Germanwings airline crashes in southern France, officials say - @Reuters http://t.co/by471NAzFg", '"@LePoint: #CharlieHebdo : "The cartoonists Charb & cabu are dead." http://t.co/7USD83cQd2 http://t.co/WGHbLiLFoX"', 'Paris #CharlieHebdo attack: This is the worst act of ter

In [15]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Inspect the data
print(train_data.head())
print(train_data.columns)

                                                text  label
0  Already, the mayor of Green Bay is having ribb...      1
1  I argued for years that we need to move from a...      1
2  The video doesn't show the broader picture of ...      0
3  A ham and cheese sandwich on one slice of brea...      0
4  He has been able to attract independents and D...      1
Index(['text', 'label'], dtype='object')


In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values


model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.6870 - loss: 0.5880 - val_accuracy: 0.8549 - val_loss: 0.3410
Epoch 2/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.9108 - loss: 0.2281 - val_accuracy: 0.8618 - val_loss: 0.3245
Epoch 3/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.9706 - loss: 0.0918 - val_accuracy: 0.8463 - val_loss: 0.4451
Epoch 4/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.9889 - loss: 0.0339 - val_accuracy: 0.8601 - val_loss: 0.4415
Epoch 5/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.9952 - loss: 0.0170 - val_accuracy: 0.8584 - val_loss: 0.7168
Epoch 6/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9943 - loss: 0.0172 - val_accuracy: 0.8618 - val_loss: 0.6284
Epoch 7/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9972 - loss: 0.0156 - val_accuracy: 0.8549 - val_loss: 0.6232
Epoch 8/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9964 - loss: 0.0146 - val_accu

In [21]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
import numpy as np

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Load GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

# Path to the GloVe file
glove_file = 'glove.6B.100d.txt'  # Change this to the actual path of the GloVe file
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = load_glove_embeddings(glove_file, word_index, embedding_dim)

# Define the CNN model with pre-trained embeddings and dropout layers
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')


C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     1,109,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,109,200 (4.23 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,109,200 (4.23 MB)

Epoch 1/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6839 - loss: 0.6187 - val_accuracy: 0.7634 - val_loss: 0.4522
Epoch 2/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7997 - loss: 0.4389 - val_accuracy: 0.7876 - val_loss: 0.4095
Epoch 3/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8223 - loss: 0.4046 - val_accuracy: 0.8238 - val_loss: 0.3692
Epoch 4/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8431 - loss: 0.3465 - val_accuracy: 0.7824 - val_loss: 0.4552
Epoch 5/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8691 - loss: 0.3047 - val_accuracy: 0.8359 - val_loss: 0.3520
Epoch 6/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8836 - loss: 0.2789 - val_accuracy: 0.8394 - val_loss: 0.3693
Epoch 7/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9090 - loss: 0.2289 - val_accuracy: 0.8497 - val_loss: 0.3688
Epoch 8/30
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9097 - loss: 0.2247 - val_accu

In [23]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, GlobalMaxPooling1D, Dense, Dropout
import numpy as np
from sklearn.model_selection import KFold

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Combine train and validation data for cross-validation
combined_data = pd.concat([train_data, val_data])

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(combined_data['text'].values)

# Convert text to sequences
X = tokenizer.texts_to_sequences(combined_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y = combined_data['label'].values
y_test = test_data['label'].values

# Load GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

# Path to the GloVe file
glove_file = 'glove.6B.100d.txt'  # Change this to the actual path of the GloVe file
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = load_glove_embeddings(glove_file, word_index, embedding_dim)

# Function to create the Bidirectional LSTM model
def create_bilstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False))
    model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_bilstm_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
    
    loss, accuracy = model.evaluate(X_val, y_val)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f'Average Cross-Validation Accuracy: {average_accuracy*100:.2f}%')

# Evaluate the final model on the test set
model = create_bilstm_model()
model.fit(X, y, epochs=10, batch_size=32)
loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')


Epoch 1/10


C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 142ms/step - accuracy: 0.7221 - loss: 0.5406 - val_accuracy: 0.7821 - val_loss: 0.4562
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 144ms/step - accuracy: 0.8143 - loss: 0.4096 - val_accuracy: 0.7793 - val_loss: 0.4486
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 143ms/step - accuracy: 0.8254 - loss: 0.3793 - val_accuracy: 0.8119 - val_loss: 0.4094
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 143ms/step - accuracy: 0.8585 - loss: 0.3342 - val_accuracy: 0.8253 - val_loss: 0.3896
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.8745 - loss: 0.2903 - val_accuracy: 0.8388 - val_loss: 0.3873
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 155ms/step - accuracy: 0.9105 - loss: 0.2265 - val_accuracy: 0.8436 - val_loss: 0.3842
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 155ms/step - accuracy: 0.9322 - loss: 0.1717 - val_accuracy: 0.8388 - val_loss: 0.3695
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 159ms/step - accuracy: 0.9544 - loss: 0.1360 - val

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('validation.csv')
test_data = pd.read_csv('test.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

model.safetensors:  14%|#4        | 62.9M/440M [00:00<?, ?B/s]